In [ ]:
from PIL import Image
import numpy as np
from scipy.ndimage import convolve
import matplotlib.pyplot as plt

# ========== 可调参数 ==========
gray_threshold = 100
horizontal_kernel_width = 100
line_response_threshold = 10
min_black_pixel_ratio = 0.1  # 每行黑色像素占比 > 50%
# ===========================
image_path = "./sample/sample1.png"
# === 图像预处理 ===
img = Image.open(image_path).convert('L')
img_np = np.array(img)
binary = (img_np < gray_threshold).astype(np.uint8)
horizontal_kernel = np.ones((1, horizontal_kernel_width), dtype=np.uint8)
response = convolve(binary, horizontal_kernel)
line_candidates = (response > line_response_threshold).astype(np.uint8)

# === 添加：黑像素数判断 ===
image_width = img_np.shape[1]
min_black_pixels_per_row = int(min_black_pixel_ratio * image_width)

# 只有这两条件都满足：卷积响应强 + 黑色像素多，才认为是五线谱线
valid_line_y_coords = [
    y for y in range(binary.shape[0])
    if np.max(line_candidates[y]) > 0 and np.sum(binary[y]) >= min_black_pixels_per_row
]

# === 可视化 ===
output_rgb = np.stack([img_np]*3, axis=-1)
for y in valid_line_y_coords:
    output_rgb[y, :, 0] = 255
    output_rgb[y, :, 1] = 0
    output_rgb[y, :, 2] = 0

plt.figure(figsize=(12, 6))
plt.imshow(output_rgb)
plt.title("Staff Lines with Black Pixel Length Filter (>50%)")
plt.axis("off")
plt.show()

In [ ]:
from PIL import Image
import numpy as np
from scipy.ndimage import convolve
import matplotlib.pyplot as plt

def detect_staff_lines(input_path, output_path, 
                       gray_threshold=100, horizontal_kernel_width=100, 
                       line_response_threshold=10, min_black_pixel_ratio=0.5):
    
    # === 图像预处理 ===
    img = Image.open(input_path).convert('L')
    img_np = np.array(img)
    binary = (img_np < gray_threshold).astype(np.uint8)

    # 卷积检测水平线
    horizontal_kernel = np.ones((1, horizontal_kernel_width), dtype=np.uint8)
    response = convolve(binary, horizontal_kernel)
    line_candidates = (response > line_response_threshold).astype(np.uint8)

    # 黑色像素数量过滤
    image_width = img_np.shape[1]
    min_black_pixels_per_row = int(min_black_pixel_ratio * image_width)

    valid_line_y_coords = [
        y for y in range(binary.shape[0])
        if np.max(line_candidates[y]) > 0 and np.sum(binary[y]) >= min_black_pixels_per_row
    ]

    # 可视化 & 保存结果
    output_rgb = np.stack([img_np]*3, axis=-1)
    for y in valid_line_y_coords:
        output_rgb[y, :, 0] = 255  # Red
        output_rgb[y, :, 1] = 0
        output_rgb[y, :, 2] = 0

    # 保存图像
    result_img = Image.fromarray(output_rgb)
    result_img.save(output_path)
    print(f"[✓] Staff lines saved to {output_path}")


In [ ]:
import os
# === 批量处理脚本 ===
input_dir = "./ds2_dense/images/train"
output_dir = "./staffline_detected"
os.makedirs(output_dir, exist_ok=True)

# 支持的图像扩展名
valid_exts = {".png", ".jpg", ".jpeg", ".bmp"}

for filename in os.listdir(input_dir):
    if any(filename.lower().endswith(ext) for ext in valid_exts):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)
        detect_staff_lines(input_path, output_path)